<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

<a id="section-one"></a>
# Задача

Прогнозирование стоимости автомобиля по характеристикам

Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE

В данном ноутбуке мы загрузили датасет, который был подготовлен на предыдущих этапах. 
Также в этом проекте мы использовали:
- 1этап:
    Ноутбук, через который парсили https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parsers-data
    Спарсенный датасет https://www.kaggle.com/juliadeinego/data-car-sales
- 2 этап:
    Ноутбук, в котором провели EDA https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parser-eda
    Датасет, после EDA 
- 3 этап:
    Ноутбук, в котором провели обучение https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parsers-ml

In [191]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import ipywidgets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from catboost import CatBoostRegressor
from catboost import cv, Pool

from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
import xgboost as xgb

from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

In [134]:
#!jupyter nbextension enable --py widgetsnbextension

In [135]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.20.1


In [136]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [137]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [138]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [139]:
VERSION    = 16
#DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключила к ноутбуку сет с пред шага
#DIR_TEST   = '../input/sf-dst-car-price-prediction/'
DIR_TRAIN = DIR_TEST = './data/' #на каггле исправить
VAL_SIZE   = 0.20   # 20%

# Data

In [140]:
!ls '../input'

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [141]:
db_source = pd.read_csv(DIR_TRAIN+'cdf_auto.csv') # датасет для обучения модели
# test = pd.read_csv(DIR_TEST+'test.csv') #на каггле исправить
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

#Выбранные признаки в EDA
data_columns = pd.read_csv(DIR_TRAIN+'cdf_auto_columns.csv') 

cat_cols = data_columns[data_columns.column_type == 'cat'].column_name.values.tolist()
bin_cols = data_columns[data_columns.column_type == 'bin'].column_name.values.tolist()
num_cols = data_columns[data_columns.column_type == 'num'].column_name.values.tolist()
target_cols = data_columns[data_columns.column_type == 'target'].column_name.values.tolist()
add_cols = data_columns[data_columns.column_type == 'add'].column_name.values.tolist()


In [142]:
data_columns

,column_name,column_type
0,description,num
1,enginePower,num
2,mileage,num
3,productionDate,num
4,mileage_per_age,num
5,time_bw_model_product,num
6,description_isD,bin
7,fuelType_isGE,bin
8,Руль,bin
9,bodyType,cat


In [143]:
print("Трейн-данных: ", len(db_source[db_source.train == 1]))
print("Тест-данных: ", len(db_source[db_source.train == 0]))
#Трейн-данных:  133055
#Тест-данных:  34686

Трейн-данных:  133055
Тест-данных:  34686


In [144]:
# датафрейм для сбора результатов по каждому экспериметну
df_Result = pd.DataFrame(columns=['model', 'MAPE'])

In [145]:
len(db_source.columns)

32

## Label Encoding

In [146]:
db_le = db_source.copy()
for colum in cat_cols + bin_cols:
    db_le[colum] = db_source[colum].astype('category').cat.codes
# db_le = database_Label Encoding

In [147]:
db_le

,bodyType,brand,color,description,enginePower,fuelType,mileage,model_name,productionDate,vehicleTransmission,...,enginePower_cut,fuelType_isGE,mileage_cut,productionDate_cut,mileage_per_age,time_bw_model_product,top2_bodyType,rare_bodyType,top5_colors,rare_colors
0,6,8,13,145,105.0,0,74000,480,2014.0,2,...,0,0,0,3,10571,1.0,0,0,1,0
1,6,8,15,357,110.0,0,60563,480,2017.0,1,...,0,0,0,3,15140,0.0,0,0,0,0
2,6,8,12,145,152.0,0,88000,692,2014.0,2,...,0,0,0,3,12571,1.0,0,0,1,0
3,6,8,6,238,110.0,0,95000,480,2014.0,0,...,0,0,0,3,13571,1.0,0,0,0,0
4,6,8,1,322,152.0,0,58536,480,2012.0,0,...,0,0,0,3,6504,4.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167736,17,2,4,0,124.0,4,91000,270,2011.0,0,...,0,0,0,3,9100,4.0,0,0,0,0
167737,17,2,13,123,124.0,4,125000,270,2010.0,0,...,0,0,0,3,11363,3.0,0,0,1,0
167738,17,2,12,46,124.0,4,119000,270,2011.0,0,...,0,0,0,3,11900,4.0,0,0,1,0
167739,11,2,1,47,161.0,4,37000,167,2018.0,0,...,0,0,0,3,12333,2.0,1,0,1,0


In [148]:
X = db_le.query('train == 1').drop(add_cols + target_cols, axis=1)#train
X_sub = db_le.query('train == 0').drop(add_cols + target_cols, axis=1) #test
y = db_source[db_source.train == 1]['price'] # train_target

In [149]:
print("Трейн-данных: ", len(X))
print("Тест-данных: ", len(X_sub))
print("Трейн-целевая: ", len(y))
# Трейн-данных:  133055
#Тест-данных:  34686
#Трейн-целевая:  133055


Трейн-данных:  133055
Тест-данных:  34686
Трейн-целевая:  133055


In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
# X_train, X_test, y_train, y_test - только Label Encoding

In [151]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(106444, 28) (26611, 28) (106444,) (26611,)


In [152]:
X.shape

(133055, 28)

## + MinMaxScaler

In [153]:
db_sc = db_le.copy()
db_sc[num_cols] = pd.DataFrame(MinMaxScaler().fit_transform(db_sc[num_cols].values), \
                                     columns = db_sc[num_cols].columns)

In [154]:
X_sc = db_sc.query('train == 1').drop(add_cols + target_cols, axis=1)#train
X_sc_sub = db_sc.query('train == 0').drop(add_cols + target_cols, axis=1) #test

In [155]:
X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(X_sc, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
# X_train_sc, X_test_sc, y_train_sc, y_test_sc - Label Encoding + MinMaxScaler

In [156]:
print(X_train_sc.shape, X_test_sc.shape, y_train_sc.shape, y_test_sc.shape)

(106444, 28) (26611, 28) (106444,) (26611,)


## + Dummies

In [157]:
db_sc_dumm = pd.get_dummies(db_sc, columns = cat_cols, dummy_na=False)

In [158]:
X_sc_dumm = db_sc_dumm.query('train == 1').drop(add_cols + target_cols, axis=1)#train
X_sc_dumm_sub = db_sc_dumm.query('train == 0').drop(add_cols + target_cols, axis=1) #test

In [159]:
X_train_sc_dumm, X_test_sc_dumm, y_train_sc_dumm, y_test_sc_dumm = train_test_split(X_sc_dumm, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
# X_train_sc_dumm, X_test_sc_dumm, y_train_sc_dumm, y_test_sc_dumm - Label Encoding + MinMaxScaler+dummies

In [160]:
print(X_train_sc_dumm.shape, X_test_sc_dumm.shape, y_train_sc_dumm.shape, y_test_sc_dumm.shape)

(106444, 915) (26611, 915) (106444,) (26611,)


# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (enginePower). 
C ней будем сравнивать другие модели.




In [161]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [162]:
"""# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median())

#оцениваем точность
result = round((mape(y_test, predict.values))*100,2)
df_Result.loc[len(df_Result)] = ['naiv', result]
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")
# Точность наивной модели по метрике MAPE: 60.66%"""

'# Находим median по экземплярам engineDisplacement в трейне и размечаем тест\npredict = X_test[\'enginePower\'].map(tmp_train.groupby(\'enginePower\')[\'price\'].median())\n\n#оцениваем точность\nresult = round((mape(y_test, predict.values))*100,2)\ndf_Result.loc[len(df_Result)] = [\'naiv\', result]\nprint(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")\n# Точность наивной модели по метрике MAPE: 60.66%'

# Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

**Полезные видео о CatBoost (на русском):**
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Default - 14.50%

In [163]:

"""catb_def = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
catb_def.fit(X_train, y_train,
         #cat_features= cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         plot=True
         )

catb_def.save_model('catboost_single_model_baseline.model')"""


"catb_def = CatBoostRegressor(iterations = 5000,\n                          random_seed = RANDOM_SEED,\n                          eval_metric='MAPE',\n                          custom_metric=['R2', 'MAE'],\n                          silent=True,\n                         )\ncatb_def.fit(X_train, y_train,\n         #cat_features= cat_features_ids,\n         eval_set=(X_test, y_test),\n         verbose_eval=0,\n         use_best_model=True,\n         plot=True\n         )\n\ncatb_def.save_model('catboost_single_model_baseline.model')"

In [164]:

"""# оцениваем точность
predict_test = catb_def.predict(X_test)

result = round((mape(y_test, predict_test))*100,2)
df_Result.loc[len(df_Result)] = ['catb_def', result]

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
# Точность модели по метрике MAPE: 14.50%
"""

'# оцениваем точность\npredict_test = catb_def.predict(X_test)\n\nresult = round((mape(y_test, predict_test))*100,2)\ndf_Result.loc[len(df_Result)] = [\'catb_def\', result]\n\nprint(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")\n# Точность модели по метрике MAPE: 14.50%\n'

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 14.54% (baseline 18%)!

## Log Target - 12.69%
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    

In [165]:
"""catb_log = CatBoostRegressor(iterations = 5000,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE', # улучшаемая метрика
                                   custom_metric=['R2', 'MAE'], # метрика для отображения на графике
                                   silent=True,
                                  )
catb_log.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

catb_log.save_model('catboost_single_model_2_baseline.model')"""

"catb_log = CatBoostRegressor(iterations = 5000,\n                                   random_seed = RANDOM_SEED,\n                                   eval_metric='MAPE', # улучшаемая метрика\n                                   custom_metric=['R2', 'MAE'], # метрика для отображения на графике\n                                   silent=True,\n                                  )\ncatb_log.fit(X_train, np.log(y_train),\n         #cat_features=cat_features_ids,\n         eval_set=(X_test, np.log(y_test)),\n         verbose_eval=0,\n         use_best_model=True,\n         #plot=True\n         )\n\ncatb_log.save_model('catboost_single_model_2_baseline.model')"

In [166]:
"""predict_test = np.exp(catb_log.predict(X_test))
#predict_submission = np.exp(model_catb_log.predict(X_sub))"""

'predict_test = np.exp(catb_log.predict(X_test))\n#predict_submission = np.exp(model_catb_log.predict(X_sub))'

In [167]:
"""result = round((mape(y_test, predict_test))*100,2)
df_Result.loc[len(df_Result)] = ['catb_log', result]

print(f"Точность модели по метрике MAPE: {result}%")"""

'result = round((mape(y_test, predict_test))*100,2)\ndf_Result.loc[len(df_Result)] = [\'catb_log\', result]\n\nprint(f"Точность модели по метрике MAPE: {result}%")'

## Log_AdvPar - 12.58%

In [168]:
"""catb_log_adv = CatBoostRegressor(iterations = 6000,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE', # улучшаемая метрика
                                   custom_metric=['R2', 'MAE'],
                                   silent=True,
                                   learning_rate = 0.05,
                                   l2_leaf_reg = 2, # Coefficient at the L2 
                                   od_type = 'Iter', # детектор переобучения
                                   od_wait = 20 # детектор переобучения
                                  )
catb_log_adv.fit(X_train, np.log(y_train),
         # cat_features=cat_cols + bin_cols, #12.78%
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

catb_log_adv.save_model('catboost_single_model_3_baseline.model')"""

"catb_log_adv = CatBoostRegressor(iterations = 6000,\n                                   random_seed = RANDOM_SEED,\n                                   eval_metric='MAPE', # улучшаемая метрика\n                                   custom_metric=['R2', 'MAE'],\n                                   silent=True,\n                                   learning_rate = 0.05,\n                                   l2_leaf_reg = 2, # Coefficient at the L2 \n                                   od_type = 'Iter', # детектор переобучения\n                                   od_wait = 20 # детектор переобучения\n                                  )\ncatb_log_adv.fit(X_train, np.log(y_train),\n         # cat_features=cat_cols + bin_cols, #12.78%\n         eval_set=(X_test, np.log(y_test)),\n         verbose_eval=0,\n         use_best_model=True,\n         #plot=True\n         )\n\ncatb_log_adv.save_model('catboost_single_model_3_baseline.model')"

In [169]:
"""print(catb_log_adv.tree_count_)"""

'print(catb_log_adv.tree_count_)'

In [170]:
"""predict_test = np.exp(catb_log_adv.predict(X_test))
#predict_submission = np.exp(model_catb_log.predict(X_sub)

result = round((mape(y_test, predict_test))*100,2)
df_Result.loc[len(df_Result)] = ['catb_log_adv', result]"""

"predict_test = np.exp(catb_log_adv.predict(X_test))\n#predict_submission = np.exp(model_catb_log.predict(X_sub)\n\nresult = round((mape(y_test, predict_test))*100,2)\ndf_Result.loc[len(df_Result)] = ['catb_log_adv', result]"

In [171]:
"""print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
# learning_rate = 0.1 12.80% (l2_leaf_reg = 7)
# learning_rate = 0.05 12.64% (l2_leaf_reg = 7)
# l2_leaf_reg = 3 12.63%, l2_leaf_reg = 3 12.60% l2_leaf_reg = 1 12.59% l2_leaf_reg = 0 12.56%"""

'print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")\n# learning_rate = 0.1 12.80% (l2_leaf_reg = 7)\n# learning_rate = 0.05 12.64% (l2_leaf_reg = 7)\n# l2_leaf_reg = 3 12.63%, l2_leaf_reg = 3 12.60% l2_leaf_reg = 1 12.59% l2_leaf_reg = 0 12.56%'

In [172]:
"""cat_cur_imp = pd.Series(catb_log_adv.get_feature_importance(),
                         X_test.columns)
fig, ax = plt.subplots(figsize=(16,14))
cat_cur_imp.plot.bar(ax=ax)
ax.set_title("Важность признаков")
fig.tight_layout()"""

'cat_cur_imp = pd.Series(catb_log_adv.get_feature_importance(),\n                         X_test.columns)\nfig, ax = plt.subplots(figsize=(16,14))\ncat_cur_imp.plot.bar(ax=ax)\nax.set_title("Важность признаков")\nfig.tight_layout()'

## Log_AdvPar_KFold - 12.57%

In [174]:
catb_log_adv_kf = CatBoostRegressor(
                                  iterations = 5000,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE', # улучшаемая метрика
                                   custom_metric=['R2', 'MAE'],
                                   silent=True,
                                   learning_rate = 0.1,
                                   l2_leaf_reg = 2, # Coefficient at the L2 
                                   od_type = 'Iter', # детектор переобучения
                                   od_wait = 20 # детектор переобучения
                          )

skf = KFold(n_splits=4, random_state=RANDOM_SEED, shuffle=True)
cat_boost_log_mape_values = []

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    
    catb_log_adv_kf.fit(X_train, np.log(y_train),
          #cat_features=cat_features_ids,
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          #plot=True
          )

    y_pred = np.exp(catb_log_adv_kf.predict(X_test))

    cat_boost_log_mape_value = mape(y_test, y_pred)
    cat_boost_log_mape_values.append(cat_boost_log_mape_value)
    print(cat_boost_log_mape_value)

print(f"Точность модели по метрике MAPE: {(np.mean(cat_boost_log_mape_values) * 100):0.2f}%.")

TRAIN: [     1      2      5 ... 133052 133053 133054] TEST: [     0      3      4 ... 133045 133046 133050]
0.12656075241234904
TRAIN: [     0      2      3 ... 133048 133050 133054] TEST: [     1      7      8 ... 133051 133052 133053]
0.12703999668135263
TRAIN: [     0      1      2 ... 133052 133053 133054] TEST: [     6      9     10 ... 133032 133033 133048]
0.12615666969671505
TRAIN: [     0      1      3 ... 133051 133052 133053] TEST: [     2      5     14 ... 133039 133041 133054]
0.12444241342340243
Точность модели по метрике MAPE: 12.60%.


In [177]:
"""result = round((np.mean(cat_boost_log_mape_values) * 100),2)
df_Result.loc[len(df_Result)] = ['catb_log_adv_kf', result]"""

Промежуточные выводы по модели CatBoostRegressor:
1. С дефолтными параметрами на базе после LabelEncoding дает результат 14.54%
2. Логорифмирование целевой базы улучшило обучение +1.8% и дошло до 12.74%. В дальнейшем буду использовать обучение с логорифмированной целевой переменной;
3. Подбор параметров немного улучшил точность +0.16% и дошло до 12.58. Подкручены были параметры:                          learning_rate = 0.1, l2_leaf_reg = 7, # Coefficient at the L2, od_type = 'Iter', # детектор переобучения
   od_wait = 20 # детектор переобучения
4. Самое интересное что для CatBoostRegressor такие штуки как LabelEncoding, MinMaxScaler, getDummies, KFold не дали никакого улучшения. 
5. Таким образом если датасет содержит большое количество категориальных данных можно после EDA+FE сразу подложить в обучение, подкрутив параметры качества (скорости) в зависимости от цели. 

# Model 3 : RandomForestRegressor
Целевая переменная в дальнейшем всегда логорифмирована

## DefPar -13.51%.

In [42]:
"""rf_def = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_def.fit(X_train, np.log(y_train))
predict_rf_def = np.exp(rf_def.predict(X_test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_rf_def) * 100):0.2f}%.")
result = round((mape(y_test, predict_rf_def) * 100),2)
#df_Result.loc[len(df_Result)] = ['rf_def', result]
# Точность модели по метрике MAPE: 13.51%."""

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s


Точность модели по метрике MAPE: 13.51%.


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.8s finished


## AdvPar - 13.58%

In [175]:
# Подбор параметров для Random forest

random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2001, num = 50)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 111, num = 20)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4]
              }

rf_adv = RandomForestRegressor(random_state=RANDOM_SEED)
rf_random = RandomizedSearchCV(estimator=rf_adv, param_distributions=random_grid, n_iter=100, 
                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
rf_random.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf_random.predict(X_test))))*100:0.2f}%")

#Fitting 3 folds for each of 100 candidates, totalling 300 fits
#Точность модели по метрике MAPE: 13.58%


Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Program Files\Python\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.88751067        nan 0.91036477 0.9096488  0.90370448
 0.9072292  0.90467219 0.9110299  0.90717661 0.90640999 0.91071916
 0.90311784 0.88721304 0.9077328  0.90438426 0.90671113 0.91067871
 0.90968595 0.91113396 0.90725868 0.91118205 0.90013041 0.90946044
 0.91116849 0.90911535 0.90452872 0.90732292 0.90332429 0.9105037
 0.90637488 0.91032292 0.90647064 0.90770491 0.90761739 0.90928986
 0.90941429 0.90941429 0.91068892 0.90427671 0.90966403 0.91084656
 0.90866602 0.91071625 0.91075095 0.90866909 0.90557752 0.90942541
 0.91027763 0.90364574 0.91083453 0.90454894 0.90441596 0.90770491
 0.90941174 0.90735916 0.91044194 0.89988397 0.90772617 0.91073476
 0.90638262 0.90926789 0.91026015 0.90926789 0.91072366 0.91040343
 0.90962622 0.91113451 0.90971495 0.91089571 0.90467163 0.91087149
 0.91094302 0.90394181 0.90424937 0.91080374 

Точность модели по метрике MAPE: 13.10%


In [176]:
print(rf_random.best_params_)
print(rf_random.best_score_)
print(rf_random.best_estimator_)
#{'n_estimators': 1800, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
#0.9004041491819783
#RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
#                      min_samples_split=5, n_estimators=1800, random_state=42)
#----------------
#{'n_estimators': 1927, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 25}
#0.9111820501392165
#RandomForestRegressor(max_depth=25, n_estimators=1927, random_state=42)

model = rf_random.best_estimator_
# другой вариант распечать подобранные параметры
#best_parameters = model.get_params()
#for param_name in sorted(best_parameters.keys()):
#        print('\t%s: %r' % (param_name, best_parameters[param_name]))

print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(model.predict(X_test))))*100:0.2f}%")
# Точность модели по метрике MAPE: 13.58%"""

{'n_estimators': 1927, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 25}
0.9111820501392165
RandomForestRegressor(max_depth=25, n_estimators=1927, random_state=42)
Точность модели по метрике MAPE: 13.10%


In [44]:
"""# данные после LabelEncoding с подобранными параметрами
rf_adv = RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_split=5, n_estimators=1800, random_state=42)
rf_adv.fit(X_train, np.log(y_train))
predict_rf_adv = np.exp(rf_adv.predict(X_test))

result = round((mape(y_test, predict_rf_adv) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
df_Result.loc[len(df_Result)] = ['rf_adv', result]
#Точность модели по метрике MAPE: 13.58%.
# что подобранные параметры ухудшили показатели, печально. Дальнейший эсперементы буду проводить с дефолтными данными"""

'# данные после LabelEncoding с подобранными параметрами\nrf_adv = RandomForestRegressor(bootstrap=False, max_depth=20, max_features=\'sqrt\',\n                      min_samples_split=5, n_estimators=1800, random_state=42)\nrf_adv.fit(X_train, np.log(y_train))\npredict_rf_adv = np.exp(rf_adv.predict(X_test))\n\nresult = round((mape(y_test, predict_rf_adv) * 100),2)\nprint(f"Точность модели по метрике MAPE: {result}%.")\ndf_Result.loc[len(df_Result)] = [\'rf_adv\', result]\n#Точность модели по метрике MAPE: 13.58%.\n# что подобранные параметры ухудшили показатели, печально. Дальнейший эсперементы буду проводить с дефолтными данными'

## DefPar_Stan_Dumm - 13.34 %

In [52]:
"""# данные после LabelEncoding, MinMaxScaler и подобранные параметры
rf_adv_stan = RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_split=5, n_estimators=1800, random_state=42)
rf_adv_stan.fit(X_train_sc, np.log(y_train_sc))
predict_rf_adv = np.exp(rf_adv_stan.predict(X_test_sc))
result = round((mape(y_test_sc, predict_rf_adv) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
#df_Result.loc[len(df_Result)] = ['rf_log_adv', result]
# Точность модели по метрике MAPE: 13.81%."""

In [54]:
"""# данные после LabelEncoding, MinMaxScaler и дефолтные параметры
rf_def_stan = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_def_stan.fit(X_train_sc, np.log(y_train_sc))
predict_rf_def_stan = np.exp(rf_def_stan.predict(X_test_sc))
result = round((mape(y_test_sc, predict_rf_def_stan) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
#df_Result.loc[len(df_Result)] = ['rf_log_adv', result]
# Точность модели по метрике MAPE: 13.51%."""

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   39.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s


Точность модели по метрике MAPE: 13.51%.


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


In [55]:
# данные после LabelEncoding,  MinMaxScaler, getDummies и дефолтные параметры
rf_def_stan_dumm = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_def_stan_dumm.fit(X_train_sc_dumm, np.log(y_train_sc_dumm))
predict_rf_def_stan_dumm = np.exp(rf_def_stan_dumm.predict(X_test_sc_dumm))

result = round((mape(y_test_sc_dumm, predict_rf_def_stan_dumm) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
df_Result.loc[len(df_Result)] = ['rf_def_stan_dumm', result]
# Точность модели по метрике MAPE: 13.34%.

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s


Точность модели по метрике MAPE: 13.34%.


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.6s finished


Промежуточные выводы по модели RandomForestRegressor:
1. Модель показала себя хуже чем CatBoost, лучший результат 13.34%.
2. Хорошо при обучении себя провила стандартизация данных (+0.3%) и даммикодирование (+0.17)

In [56]:
df_Result

,model,MAPE
0,naiv,63.27
1,catb_log_adv,12.58
2,rf_def,13.51
3,rf_def_stan_dumm,13.34


# Model 3 - DecisionTreeRegressor

## DefPar - 17.12%

In [32]:
dt_def = DecisionTreeRegressor(max_depth=15, random_state = RANDOM_SEED)
dt_def.fit(X_train, np.log(y_train))
predict_dt_def = np.exp(dt_def.predict(X_test))
result = round((mape(y_test, predict_dt_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 17.12%.

Точность модели по метрике MAPE: 17.12%.


## DefPar_Stan -  17.12%

In [31]:
# данные стандартизированы, дефолтные параметры
dt_def_sc = DecisionTreeRegressor(max_depth=15, random_state = RANDOM_SEED)
dt_def_sc.fit(X_train_sc, np.log(y_train_sc))
predict_dt_def_sc = np.exp(dt_def_sc.predict(X_test_sc))
result = round((mape(y_test_sc, predict_dt_def_sc) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 17.12%.

Точность модели по метрике MAPE: 17.12%.


In [65]:
"""# данные стандартизированы + даммизированы, дефолтные параметры
dt_def_sc_dumm = DecisionTreeRegressor(max_depth=15, random_state = RANDOM_SEED)
dt_def_sc_dumm.fit(X_train_sc_dumm, np.log(y_train_sc_dumm))
predict_dt_def_sc_dumm = np.exp(dt_def_sc_dumm.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict_dt_def_sc_dumm) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 17.28%."""

Точность модели по метрике MAPE: 17.28%.


## AdvPar_Stan

In [42]:
"""parameters = {'splitter':('best','random'), 
              'max_depth':np.arange(10,20), 
              'min_samples_split':np.arange(2,10), 
              'min_samples_leaf':np.arange(2,10)}

regressor = GridSearchCV(DecisionTreeRegressor(), parameters, cv = 3)
regressor.fit(X_train_sc, np.log(y_train_sc+1))

print('Best score: ', regressor.best_score_)
print(regressor.best_params_)
dt_adv_sc = regressor.best_estimator_
predict_dt_adv_sc = np.exp(dt_adv_sc.predict(X_test_sc))
result = round((mape(y_test_sc, predict_dt_adv_sc) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")"""

'parameters = {\'splitter\':(\'best\',\'random\'), \n              \'max_depth\':np.arange(10,20), \n              \'min_samples_split\':np.arange(2,10), \n              \'min_samples_leaf\':np.arange(2,10)}\n\nregressor = GridSearchCV(DecisionTreeRegressor(), parameters, cv = 3)\nregressor.fit(X_train_sc, np.log(y_train_sc+1))\n\nprint(\'Best score: \', regressor.best_score_)\nprint(regressor.best_params_)\ndt_adv_sc = regressor.best_estimator_\npredict_dt_adv_sc = np.exp(dt_adv_sc.predict(X_test_sc))\nresult = round((mape(y_test_sc, predict_dt_adv_sc) * 100),2)\nprint(f"Точность модели по метрике MAPE: {result}%.")'

In [74]:
print(regressor.best_params_)

{'max_depth': 9, 'min_samples_leaf': 4, 'min_samples_split': 9, 'splitter': 'best'}


Промежуточные выводы по DecisionTreeRegressor:
1. Точность предсказания плохая по сравнению с другими моделями.
2. Стандартизирование не улучшило ситуацию, даммизирование даже ухудшило.
3. Подбор лучших параметров работает долго, не буду дальше улучшать эту модель..

# Model 4 - ExtraTreesRegressor

## DefPar - 13.82%

In [34]:
etr_def = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_def.fit(X_train, np.log(y_train+1))
predict_etr_def = np.exp(etr_def.predict(X_test))
result = round((mape(y_test, predict_etr_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.82%

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s


Точность модели по метрике MAPE: 13.82%


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished


## DefPar_Stan - 13.54%

In [39]:
# данные стандартизированы, дефолтные параметры
etr_def = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_def.fit(X_train_sc, np.log(y_train_sc+1))
predict_etr_def = np.exp(etr_def.predict(X_test_sc))
result = round((mape(y_test_sc, predict_etr_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.83%

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s


Точность модели по метрике MAPE: 13.83%


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished


In [40]:
# данные стандартизированы + даммизированы, дефолтные параметры
etr_def = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_def.fit(X_train_sc_dumm, np.log(y_train_sc_dumm+1))
predict_etr_def = np.exp(etr_def.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict_etr_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.54%

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s


Точность модели по метрике MAPE: 13.54%


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.1s finished


## DefPar_KFold - 13.61%

In [43]:
#данные LabelEncoding, дефолтные параметры, KFold с n_splits=4
etr_KFold = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)

skf = KFold(n_splits=4, random_state=RANDOM_SEED, shuffle=True)
etr_mape_values = []

for train_index, test_index in skf.split(X_sc_dumm, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    
    # For training, fit() is used
    etr_KFold.fit(X_train, np.log(y_train))

    # For MAPE metric (or any other), we need the predictions of the model
    y_pred = np.exp(etr_KFold.predict(X_test))

    etr_mape_value = mape(y_test, y_pred)
    etr_mape_values.append(etr_mape_value)
    print(etr_mape_values)

print(f"Точность модели по метрике MAPE: {(np.mean(etr_mape_values) * 100):0.2f}%.")
# Точность модели по метрике MAPE: 13.61%.
# Точность модели по метрике MAPE: 13.61%. после стандартизации
# Точность модели по метрике MAPE: 13.61%. после стандартизации и даммизации

TRAIN: [     1      2      5 ... 133052 133053 133054] TEST: [     0      3      4 ... 133045 133046 133050]


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   34.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


[0.13641810126820308]
TRAIN: [     0      2      3 ... 133048 133050 133054] TEST: [     1      7      8 ... 133051 133052 133053]


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   35.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.6s finished


[0.13641810126820308, 0.13690322991652734]
TRAIN: [     0      1      2 ... 133052 133053 133054] TEST: [     6      9     10 ... 133032 133033 133048]


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   34.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


[0.13641810126820308, 0.13690322991652734, 0.13530346069310104]
TRAIN: [     0      1      3 ... 133051 133052 133053] TEST: [     2      5     14 ... 133039 133041 133054]


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   34.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s


[0.13641810126820308, 0.13690322991652734, 0.13530346069310104, 0.1356546649831521]
Точность модели по метрике MAPE: 13.61%.


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


## AdvPar_Stan_Dumm

In [219]:
space_etr = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
            'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
             }

def hyperopt_etr_score(params):
    clf = ExtraTreesRegressor(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, X, y, cv=3).mean()
    print(current_score, params)
    return -current_score
 
 
simple_space_etr = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
            'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
               }
 
best = fmin(fn=hyperopt_etr_score, space=simple_space_etr, algo=tpe.suggest, max_evals=10)
best

-1.1835837550363504                                                                                                    
{'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 860}                                   
-1.2286323173220193                                                                                                    
{'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 555}                                   
-0.31516124124557376                                                                                                   
{'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 771}                                  
-0.2621432935878934                                                                                                    
{'max_depth': 13, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 592}                                  
-0.5271010193481565                     

{'max_depth': 0,
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'n_estimators': 129}

In [221]:
#hp.uniform('n_estimators',100,500),
#hp.choice("n_estimators", [int(x) for x in np.linspace(200, 1000, num = 17)])

def objective(params):
     model=ExtraTreesRegressor(
         n_estimators=int(params['n_estimators']),
         max_depth=int(params['max_depth']),
         min_samples_leaf=int(params['min_samples_leaf']),
         min_samples_split=int(params['min_samples_split']),
         bootstrap=params['bootstrap'],
         max_features=params['max_features'],
         #random_state=RANDOM_SEED,
         n_jobs=-1,
         early_stopping_rounds = 10
     )
     model.fit(X_train, np.log(y_train))
     pred=model.predict(X_test)
     score=mape(y_test,np.exp(pred))
     return score

def optimize(trial):    
    params={
         'n_estimators': hp.choice('n_estimators', range(100, 500)),
         'max_features': hp.choice("max_features", ['auto', 'sqrt']),
         'max_depth': hp.choice('max_depth',np.arange(5,15, dtype=int)),
         'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
         'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
         'bootstrap': hp.choice("bootstrap", [True, False]),
     }
                
    best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=20)
    return best

trial=Trials()
best=optimize(trial)
best

100%|████████████████████████████████████████████████| 20/20 [04:34<00:00, 13.71s/trial, best loss: 0.1502856480972811]


In [222]:
best

{'bootstrap': 1,
 'max_depth': 9,
 'max_features': 0,
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'n_estimators': 115}

In [224]:
hp_results

NameError: name 'hp_results' is not defined

# Model 5 - GradientBoostingRegressor

## DefPar - 12.85%

In [44]:
gb_def = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb_def.fit(X_train, np.log(y_train+1))
predict_gb_def = np.exp(gb_def.predict(X_test))
result = round((mape(y_test, predict_gb_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 12.85%

Точность модели по метрике MAPE: 12.85%


## DefPar_Stan

In [52]:
# данные стандартизированы, дефолтные параметры
gb_def = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb_def.fit(X_train_sc, np.log(y_train_sc+1))
predict_gb_def = np.exp(gb_def.predict(X_test_sc))
result = round((mape(y_test_sc, predict_gb_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.0%

Точность модели по метрике MAPE: 13.0%


In [53]:
# данные стандартизированы + даммизирвоанные, дефолтные параметры
gb_def = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb_def.fit(X_train_sc_dumm, np.log(y_train_sc_dumm+1))
predict_gb_def = np.exp(gb_def.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict_gb_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.0%

KeyboardInterrupt: 

# Model 6 - xgboost

## DefPar - 12.27%

In [51]:
xb_def = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb_def.fit(X_train, np.log(y_train+1))
predict_xb_def = np.exp(xb_def.predict(X_test))
result = round((mape(y_test, predict_xb_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 12.27%

Точность модели по метрике MAPE: 12.27%


## AdvPar -

In [192]:
space_xgb2 = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            # Increase this number if you have more cores. Otherwise, remove it and it will default
            # to the maxium number.
            #'nthread': 4,
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1
        }

def hyperopt_xgb_score(params):
    clf = xgb.XGBRegressor(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, X, y, cv=3).mean()
    print(current_score, params)
    return -current_score
 
 
simple_space_xgb = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
}
 
best = fmin(fn=hyperopt_xgb_score, space=simple_space_xgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

0.7026894762150396                                                                                                     
{'eta': 0.325, 'max_depth': 5, 'n_estimators': 812}                                                                    
0.6954513086182801                                                                                                     
{'eta': 0.30000000000000004, 'max_depth': 12, 'n_estimators': 239}                                                     
0.6151461457845006                                                                                                     
{'eta': 0.2, 'max_depth': 3, 'n_estimators': 866}                                                                      
0.7653448150384431                                                                                                     
{'eta': 0.275, 'max_depth': 5, 'n_estimators': 634}                                                                    
0.8041513138194616                      

# Model 7 : StackingRegressor

In [ ]:
# estimators = [
#     ('etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
#     ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=RANDOM_SEED))
# ]

# sr_log = StackingRegressor(
#     estimators=estimators,
#     final_estimator=LinearRegression()
# )

# # For training, fit() is used
# sr_log.fit(X_train, np.log(y_train))

# # For MAPE metric (or any other), we need the predictions of the model
# y_pred = np.exp(sr_log.predict(X_test))

# print(f"The MAPE mertic for the default StackingRegressor model: {(mape(y_test, y_pred) * 100):0.2f}%."

# Submission

In [181]:
df_Result

,model,MAPE
0,naiv,63.27
1,catb_def,14.54
2,catb_log,12.74
3,catb_log_adv,12.58
4,catb_log_adv_kf,12.57
5,rf_log,13.29


In [180]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

NameError: name 'predict_submission' is not defined

## Итоговая МАРЕ 13.5% на лидерборде 12.391. Место на Kaggle 26.01.2002 - 33

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

In [132]:

db_source[db_source.new_data == 0].head(5)

,bodyType,brand,color,description,enginePower,fuelType,mileage,model_name,productionDate,vehicleTransmission,...,enginePower_cut,fuelType_isGE,mileage_cut,productionDate_cut,mileage_per_age,time_bw_model_product,top2_bodyType,rare_bodyType,top5_colors,rare_colors
0,лифтбек,skoda,синий,145,105.0,бензин,74000,octavia,2014.0,ROBOT,...,1,0,1,4,10571,1.0,0,0,1,0
1,лифтбек,skoda,чёрный,357,110.0,бензин,60563,octavia,2017.0,MECHANICAL,...,1,0,1,4,15140,0.0,0,0,0,0
2,лифтбек,skoda,серый,145,152.0,бензин,88000,superb,2014.0,ROBOT,...,1,0,1,4,12571,1.0,0,0,1,0
3,лифтбек,skoda,коричневый,238,110.0,бензин,95000,octavia,2014.0,AUTOMATIC,...,1,0,1,4,13571,1.0,0,0,0,0
4,лифтбек,skoda,белый,322,152.0,бензин,58536,octavia,2012.0,AUTOMATIC,...,1,0,1,4,6504,4.0,0,0,1,0
